In [1]:
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
from bson import ObjectId
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sshtunnel import SSHTunnelForwarder
from dbutils.pooled_db import PooledDB

ssh_host = 'api.trustyapp.io'
ssh_username = 'ubuntu'
ssh_pem_file = './ng-prod.pem'
database_host = '127.0.0.1'  # SSH 터널을 통해 localhost로 접근
database_name = 'trusty'
database_username = 'server'
database_password = 'ygFAhsyTMDzB21VdLTnoq7djxwjDNzuc9yPGukm73otXER8ihD'

server = SSHTunnelForwarder(
    (ssh_host, 22),  # SSH 서버 주소와 포트
    ssh_username=ssh_username,
    ssh_pkey=ssh_pem_file,  # Private key 파일 경로
    remote_bind_address=(database_host, 5432)  # 원격 MySQL 서버 주소와 포트
)

server.start()

conn = psycopg2.connect(
    host='127.0.0.1',  # localhost를 통해 SSH 터널로 접속
    user=database_username,
    password=database_password,
    database=database_name,
    port=server.local_bind_port  # sshtunnel이 자동으로 할당한 포트
)
cursor = conn.cursor()

In [5]:
conn.rollback()

qry = """
SELECT *
FROM credit_history ch
WHERE AGE(ch."dueAt", ch."openedAt") <= INTERVAL '15 days';
        """
cursor.execute(qry)
result = cursor.fetchall()

# DataFrame 생성
df = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])
df.to_csv('평균대출금액_파_15.csv', index=False, encoding='utf-8-sig')

# 결과 확인
print(df.head())

                         createdAt                        updatedAt    id  \
0 2025-02-13 11:33:04.257602+00:00 2025-02-13 11:33:04.257602+00:00  7145   
1 2025-02-13 11:33:04.257602+00:00 2025-02-13 11:33:04.257602+00:00  7149   
2 2025-02-13 11:33:04.257602+00:00 2025-02-13 11:33:04.257602+00:00  7150   
3 2025-02-13 11:57:34.542321+00:00 2025-02-13 11:57:34.542321+00:00  7162   
4 2024-10-29 12:49:13.170622+00:00 2024-10-29 12:49:13.170622+00:00     7   

          bureau  reportedAt                       institutionName  \
0  first-central  2025-01-24                       NewEdge Finance   
1  first-central  2023-05-01                       NewEdge Finance   
2  first-central  2023-04-07                       NewEdge Finance   
3            crc  2024-02-27  ALTRACRED FINANCE INVESTMENT LIMITED   
4  first-central  2023-05-25                       NewEdge Finance   

         accountNumber                loanDescription loanAmount  \
0           1500034654  Payday Loan or salary ad